# daily KD code

## 1. Import package

In [1]:
import mysql.connector
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
# %pip install matplotlib
# import matplotlib
# import secret as sc
import pymysql
from sqlalchemy import create_engine

In [2]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://root:123456@db/iii_project", echo=False)
readout = pd.read_sql_table('eur_usd_4hr', engine)
readout.columns = ["trans_time", "open_price", "high_price", "low_price", "close_price"]
df = readout

In [3]:
df.dtypes

trans_time     datetime64[ns]
open_price            float64
high_price            float64
low_price             float64
close_price           float64
dtype: object

## 3. 資料清理

### 3.2 清除離群值

In [4]:
def rpoutrange(df):
    dff = df
    colnm = ["open_price", "high_price", "low_price", "close_price"]
    for col in colnm:
        price_std = dff.describe()[col].loc["std"]
        price_mean = dff.describe()[col].loc["mean"]
        # 尋找離群值超過6個標準差的值
        flt_out = (dff[col] > price_mean + price_std*6) | (dff[col] < price_mean - price_std*6)
        flt_out_idx = dff[flt_out].index.values.astype(int)
        dff[col].iloc[flt_out_idx] = (dff[col].iloc[flt_out_idx+1].values + dff[col].iloc[flt_out_idx-1].values) / 2
    return dff

In [5]:
df = rpoutrange(df)
df.describe()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,open_price,high_price,low_price,close_price
count,27700.000000,27700.000000,27700.000000,27700.000000
mean,1.228801,1.230904,1.226742,1.228810
std,0.166815,0.167034,0.166559,0.166804
min,0.823600,0.827900,0.822900,0.823500
25%,1.119300,1.121152,1.117545,1.119337
50%,1.261500,1.263900,1.258900,1.261500
75%,1.345900,1.347800,1.344100,1.345900
max,1.601000,1.604100,1.597700,1.601100


In [107]:
df_day = pd.DataFrame(index = tmpdf.size().index)
tmpdf = df.groupby(by = "date")
df_day["open_price"] = tmpdf.head(1).set_index("date")["open_price"]
df_day["close_price"] = tmpdf.tail(1).set_index("date")["close_price"]
df_day["high_price"] = tmpdf.max()["high_price"]
df_day["low_price"] = tmpdf.min()["low_price"]
df_day = df_day.reset_index()
df_day

,date,open_price,close_price,high_price,low_price
0,2000-05-30,0.93020,0.93150,0.93240,0.92930
1,2000-05-31,0.93150,0.93770,0.93820,0.91510
2,2000-06-01,0.93760,0.93150,0.93800,0.92870
3,2000-06-02,0.93130,0.94400,0.94980,0.93080
4,2000-06-04,0.94610,0.94430,0.94640,0.94330
...,...,...,...,...,...
5384,2017-12-25,1.18678,1.18697,1.18737,1.18513
5385,2017-12-26,1.18698,1.18745,1.18786,1.18464
5386,2017-12-27,1.18744,1.19173,1.19186,1.18644
5387,2017-12-28,1.19174,1.19443,1.19589,1.19163


## 4. 將 4hr data 轉成 1day data

### 4.1 將時間取出萃取成一天

In [6]:
df["date"] = df["trans_time"].dt.date.astype(str)
df_day = df["date"].drop_duplicates().reset_index().drop(columns=["index"])
df_day

,date
0,2000-05-30
1,2000-05-31
2,2000-06-01
3,2000-06-02
4,2000-06-04
...,...
5384,2017-12-25
5385,2017-12-26
5386,2017-12-27
5387,2017-12-28


### 4.2 定義 function 取每日的最高價、最低價、開盤價、收盤價

In [7]:
def high_d(df2, time):
    df_tmp = df2[df2["date"] == time].reset_index().drop(columns=["index", "trans_time"])
    return max(df_tmp["high_price"])

def low_d(df2, time):
    df_tmp = df2[df2["date"] == time].reset_index().drop(columns=["index", "trans_time"])
    return min(df_tmp["low_price"])

def open_d(df2, time):
    df_tmp = df2[df2["date"] == time].reset_index().drop(columns=["index", "trans_time"])
    open_price_d = df_tmp["open_price"].iloc[0]
    return open_price_d

def close_d(df2, time):
    df_tmp = df2[df2["date"] == time].reset_index().drop(columns=["index", "trans_time"])
    close_price_d = df_tmp["close_price"].iloc[len(df_tmp)-1]
    return close_price_d

### 4.3 計算每日data 放進 df_day 內

In [9]:
df_day["open_price"] = df_day["date"].apply(lambda x : open_d(df, x))
df_day["close_price"] = df_day["date"].apply(lambda x : close_d(df, x))
df_day["high_price"] = df_day["date"].apply(lambda x : high_d(df, x))
df_day["low_price"] = df_day["date"].apply(lambda x : low_d(df, x))

In [10]:
df_day

,date,open_price,close_price,high_price,low_price
0,2000-05-30,0.93020,0.93150,0.93240,0.92930
1,2000-05-31,0.93150,0.93770,0.93820,0.91510
2,2000-06-01,0.93760,0.93150,0.93800,0.92870
3,2000-06-02,0.93130,0.94400,0.94980,0.93080
4,2000-06-04,0.94610,0.94430,0.94640,0.94330
...,...,...,...,...,...
5384,2017-12-25,1.18678,1.18697,1.18737,1.18513
5385,2017-12-26,1.18698,1.18745,1.18786,1.18464
5386,2017-12-27,1.18744,1.19173,1.19186,1.18644
5387,2017-12-28,1.19174,1.19443,1.19589,1.19163


## 5. 定義 rsv function，並計算出 rsv

In [108]:
def rsv(new_df):
    rsv = []
    for i in range(len(new_df)):
        if i < 8:
            rsv.append(0)
            # print(rsv)
        else:
            highp = []
            lowp = []
            for j in range(9):
                highp.append(new_df["high_price"].iloc[i-j])
                lowp.append(new_df["low_price"].iloc[i-j])
            high_max = max(highp)
            low_min = min(lowp)
            # print(new_df["close_price"].iloc[i],high_max,low_min)
            rsv_v = 100*(new_df["close_price"].iloc[i]-low_min)/(high_max-low_min)
            rsv.append(round(rsv_v,3))
            # print(rsv)
    return rsv

In [109]:
df_day["rsv"] = pd.Series(rsv(df_day))
df_day

,date,open_price,close_price,high_price,low_price,rsv
0,2000-05-30,0.93020,0.93150,0.93240,0.92930,0.000
1,2000-05-31,0.93150,0.93770,0.93820,0.91510,0.000
2,2000-06-01,0.93760,0.93150,0.93800,0.92870,0.000
3,2000-06-02,0.93130,0.94400,0.94980,0.93080,0.000
4,2000-06-04,0.94610,0.94430,0.94640,0.94330,0.000
...,...,...,...,...,...,...
5384,2017-12-25,1.18678,1.18697,1.18737,1.18513,80.559
5385,2017-12-26,1.18698,1.18745,1.18786,1.18464,83.475
5386,2017-12-27,1.18744,1.19173,1.19186,1.18644,99.284
5387,2017-12-28,1.19174,1.19443,1.19589,1.19163,92.674


## 6. 定義 K/D 線 function，並算出 K/D 值

In [110]:
def k_lst(new_df):
    k_list = []
    for i in range(len(new_df)):
        if i == 0:
            k_list.append(0)
        else:
            k_value = (2/3) * k_list[i-1] + (1/3) * new_df["rsv"].iloc[i]
            k_list.append(round(k_value,3))
    return k_list

def d_lst(new_df):
    d_list = []
    for i in range(len(new_df)):
        if i == 0:
            d_list.append(0)
        else:
            d_value = (2/3) * d_list[i-1] + (1/3) * new_df["K_value"].iloc[i]
            d_list.append(round(d_value,3))
    return d_list

In [111]:
df_day["K_value"] = pd.Series(k_lst(df_day))
df_day["D_value"] = pd.Series(d_lst(df_day))
df_day

,date,open_price,close_price,high_price,low_price,rsv,K_value,D_value
0,2000-05-30,0.93020,0.93150,0.93240,0.92930,0.000,0.000,0.000
1,2000-05-31,0.93150,0.93770,0.93820,0.91510,0.000,0.000,0.000
2,2000-06-01,0.93760,0.93150,0.93800,0.92870,0.000,0.000,0.000
3,2000-06-02,0.93130,0.94400,0.94980,0.93080,0.000,0.000,0.000
4,2000-06-04,0.94610,0.94430,0.94640,0.94330,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...
5384,2017-12-25,1.18678,1.18697,1.18737,1.18513,80.559,72.864,63.702
5385,2017-12-26,1.18698,1.18745,1.18786,1.18464,83.475,76.401,67.935
5386,2017-12-27,1.18744,1.19173,1.19186,1.18644,99.284,84.029,73.300
5387,2017-12-28,1.19174,1.19443,1.19589,1.19163,92.674,86.911,77.837


## 7. 判斷 KD 值交叉時機

In [112]:
def kd_cross(new_df):
    new_df["K-D"] = new_df.K_value - new_df.D_value
    new_df["delt_K"] = new_df["K_value"] - new_df["K_value"].shift(1)
    k_boln_M = (new_df["K_value"] > 80) & (new_df["D_value"] > 80)
    k_boln_m = (new_df["K_value"] < 20) & (new_df["D_value"] < 20)
    kd_x = new_df["K-D"] * new_df["K-D"].shift(1) < 0
    selout = k_boln_m & kd_x
    buyin = k_boln_M & kd_x
    tradelen = len(new_df)
    trd = [0]
    for i in range(1,tradelen):
        trd.append(trd[i-1])
        if buyin[i] == True:
            trd[i] = 100
        elif selout[i] == True:
            trd[i] = 0
    new_df["KD_tunnel"] = pd.Series(trd)

In [113]:
kd_cross(df_day)
df_day

,date,open_price,close_price,high_price,low_price,rsv,K_value,D_value,K-D,delt_K,KD_tunnel
0,2000-05-30,0.93020,0.93150,0.93240,0.92930,0.000,0.000,0.000,0.000,NaN,0
1,2000-05-31,0.93150,0.93770,0.93820,0.91510,0.000,0.000,0.000,0.000,0.000,0
2,2000-06-01,0.93760,0.93150,0.93800,0.92870,0.000,0.000,0.000,0.000,0.000,0
3,2000-06-02,0.93130,0.94400,0.94980,0.93080,0.000,0.000,0.000,0.000,0.000,0
4,2000-06-04,0.94610,0.94430,0.94640,0.94330,0.000,0.000,0.000,0.000,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...
5384,2017-12-25,1.18678,1.18697,1.18737,1.18513,80.559,72.864,63.702,9.162,3.848,100
5385,2017-12-26,1.18698,1.18745,1.18786,1.18464,83.475,76.401,67.935,8.466,3.537,100
5386,2017-12-27,1.18744,1.19173,1.19186,1.18644,99.284,84.029,73.300,10.729,7.628,100
5387,2017-12-28,1.19174,1.19443,1.19589,1.19163,92.674,86.911,77.837,9.074,2.882,100


## 8. plotly 畫出圖

In [114]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

figdf2 = df_day
figdf2 = figdf2.set_index("date")
figdf2 = figdf2.loc["2016-01-01":"2017-02-30"]


# fig2 = go.Figure()
fig2 = make_subplots(specs=[[{"secondary_y": True}]])

fig2.add_trace(go.Scatter(
                x=figdf2.index,
                y=figdf2['K_value'],
                name="K_line",
                line_color='blue',
                opacity=0.3),
                secondary_y=False)

fig2.add_trace(go.Scatter(
                x=figdf2.index,
                y=figdf2['D_value'],
                name="D_line",
                line_color='red',
                opacity=0.3),
                secondary_y=False)

fig2.add_trace(go.Scatter(
                x = figdf2.index,
                y = [20]*len(figdf2),
                name="KD_lowb",
                line_color="black",
                opacity=0.5),
                secondary_y=False)

fig2.add_trace(go.Scatter(
                x = figdf2.index,
                y = [80]*len(figdf2),
                name="KD_upb",
                line_color="black",
                opacity=0.5),
                secondary_y=False)

fig2.add_trace(go.Scatter(
                x = figdf2.index,
                y=figdf2['KD_tunnel'],
                name="KD_buysell_tunnel",
                line_color="orange"),
                secondary_y=False)

fig2.add_trace(go.Candlestick(x=figdf2.index,
                open=figdf2['open_price'],
                high=figdf2['high_price'],
                low=figdf2['low_price'],
                close=figdf2['close_price'],
                name="K-line"),
                secondary_y=True)

# Use date string to set xaxis range
fig2.update_layout(title_text="Daily OHLC & KD trend chart", 
                   xaxis_rangeslider_visible=True)
fig2.show()

### 8.1 將 fig 轉成 img 輸出

- 需要 orca 軟體
    - Ref: https://github.com/plotly/orca

In [18]:
# %pip install orca

In [19]:
# %conda install psutil
# %conda install -c plotly plotly-orca
# %conda update -n base conda

In [117]:
from IPython.display import Image
Image(fig2.to_image(format="png"))

ValueError: 
For some reason plotly.py was unable to communicate with the
local orca server process, even though the server process seems to be running.

Please review the process and connection information below:

orca status
-----------
    state: running
    executable: /usr/bin/xvfb-run --auto-servernum --server-args -screen 0 640x480x24 +extension RANDR +extension GLX /opt/conda/bin/orca
    version: 1.2.1
    port: 40762
    pid: 106
    command: ['/usr/bin/xvfb-run', '--auto-servernum', '--server-args', '-screen 0 640x480x24 +extension RANDR +extension GLX', '/opt/conda/bin/orca', 'serve', '-p', '40762', '--plotly', '/opt/conda/lib/python3.7/site-packages/plotly/package_data/plotly.min.js', '--graph-only', '--mathjax', 'https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js']




### 8.2 將 fig 轉成 png 輸出

In [21]:
import os, datetime
import psutil, requests

today = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

if not os.path.exists("images"):
    os.mkdir("images")

fig2.write_image("images/KD_daily_"+today+".png")